## Applying ToMe to timm models
A quick example showing how to apply ToMe to a timm model off-the-shelf.

In [28]:
# !pip install timm
# !pip install tome
# !cd /home/hyunwoo137/research/lightweight/Colab Notebooks/ToMe-main
# !python /home/hyunwoo137/research/lightweight/Colab_Notebooks/ToMe-main/setup.py build develop
# !pip list
# !pip install tome==0.1
# import .tome
# import ../tome
import timm
import tome
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode


from PIL import Image

ModuleNotFoundError: No module named 'tome'

In [24]:
# Use any ViT model here (see timm.models.vision_transformer)
model_name = "vit_base_patch16_224"

# Load a pretrained model
model = timm.create_model(model_name, pretrained=True)

In [25]:
input_size = model.default_cfg["input_size"][1]

transform = transforms.Compose([
    transforms.Resize(int((256 / 224) * input_size), interpolation=InterpolationMode.BICUBIC),
    transforms.CenterCrop(input_size),
    transforms.ToTensor(),
    transforms.Normalize(model.default_cfg["mean"], model.default_cfg["std"]),
])

In [27]:
img = Image.open("images/husky.png")
img_tensor = transform(img)[None, ...]

img  # Note: image generated with stable diffusion

NameError: name 'Image' is not defined

In [ ]:
# Run the image through the baseline model
# 248: Eskimo dog, husky
# 249: malamute, malemute, Alaskan malamute
# 250: Siberian husky
model(img_tensor).topk(5).indices[0].tolist()

[248, 250, 249, 269, 273]

### Applying ToMe
Simply patch the model after initialization to enable ToMe.

In [ ]:
# Patch the model with ToMe
tome.patch.timm(model)

In [ ]:
# Run the model with no reduction (should be the same as before)
model.r = 0
model(img_tensor).topk(5).indices[0].tolist()

[248, 250, 249, 269, 273]

In [ ]:
# Run the model with some reduction
# Only the least applicable class changed
model.r = 8
model(img_tensor).topk(5).indices[0].tolist()

[248, 250, 249, 269, 537]

In [ ]:
# Run the model with a lot of reduction
# Top-3 most applicable classes didn't change (husky, Siberian husky, Alaskan malamute)
# But model is 2x faster now! See 1_benchmark_timm.ipynb
model.r = 16
model(img_tensor).topk(5).indices[0].tolist()

[248, 250, 249, 537, 269]